In [ ]:
import gc
import os
import shutil
import numpy as np
import tensorflow as tf
from sklearn.cross_decomposition import PLSRegression
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, UpSampling2D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Load data
path = '/content/trainingfinal5'
class_dirs = sorted([d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))])[0:18]

# Create temporary directory for test folders
temp_path = os.path.join(path, 'temp')
os.makedirs(temp_path, exist_ok=True)

def create_autoencoder():
    autoencoder = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64,64, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(3, (3, 3), activation='sigmoid', padding='same')
    ])
    return autoencoder

# Train PLS regressor
def create_pls_regressor():
    regressor = PLSRegression(n_components=15)  # Adjust the number of components based on your needs
    return regressor

# Load data
for test_dir in class_dirs:
    # Load images and labels for training and testing
    X_train, y_train = [], []
    X_test, y_test = [], []

    print(f"\nTesting on directory: {test_dir}")
    print("Training on directories: ", [d for d in class_dirs if d != test_dir])

    for class_dir in class_dirs:
        class_path = os.path.join(path, class_dir)
        label = float(class_dir.split("_")[1])

        if not os.path.isdir(class_path):
            continue

        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(64, 64))
            img_array = tf.keras.preprocessing.image.img_to_array(img)

            if class_dir == test_dir:
                X_test.append(img_array)
                y_test.append(label)
            else:
                X_train.append(img_array)
                y_train.append(label)

    # Move test folder to temporary directory
    src_dir = os.path.join(path, test_dir)
    dst_dir = os.path.join(temp_path, test_dir)
    shutil.move(src_dir, dst_dir)

    X_train = np.array(X_train) / 255.0
    y_train = np.array(y_train)
    X_test = np.array(X_test) / 255.0
    y_test = np.array(y_test)

    # Train autoencoder
    autoencoder = create_autoencoder()
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    autoencoder.fit(X_train, X_train, epochs=15, shuffle=True, validation_split=0.01)

    # Extract the trained encoder part of the autoencoder
    encoder_input = autoencoder.layers[0].input
    encoder_output = autoencoder.layers[4].output
    encoder = Model(inputs=encoder_input, outputs=encoder_output)

    X_train_encoded = encoder.predict(X_train)
    X_train_encoded = X_train_encoded.reshape(X_train_encoded.shape[0], -1)

    # Encode testing data
    X_test_encoded = encoder.predict(X_test)
    X_test_encoded = X_test_encoded.reshape(X_test_encoded.shape[0], -1)

    # Scale the data
    scaler = StandardScaler()
    X_train_encoded_scaled = scaler.fit_transform(X_train_encoded)
    X_test_encoded_scaled = scaler.transform(X_test_encoded)

    # Train PLS regressor
    pls_regressor = create_pls_regressor()
    pls_regressor.fit(X_train_encoded, y_train)

    # Make predictions
    preds = pls_regressor.predict(X_test_encoded).ravel()

    # Save the predictions for this folder
    with open(f"predictions_auto_final_ala222{test_dir}.txt", "w") as f:
        for pred, true_label in zip(preds, y_test):
            f.write(f"True label: {true_label}, Predicted value: {pred}\n")

    tf.keras.backend.clear_session()
    gc.collect()

    # Move test folder back to the original directory
    shutil.move(dst_dir, src_dir)
